# geoDB access

The geoDB is a service provided by the [Euro Data Cube project](https://eurodatacube.com) (EDC) as a payed service. It comes
with a Python client that provides high level access to your data and a certain amount of space in a PostGreSQL database. For exploring data you will need a geoDB subscription which you can purchase at the [EDC market place](https://eurodatacube.com/marketplace).

You can access the service in two ways:

- By using the Jupyter Python notebook provided by EDC Marketplace (configuration-free, ```geodb = GeoDBClient()```)
- By using your own Jupyter notebook or Python script by providing a client id and secret to the GeoDBClient (```geodb = GeoDBClient(client_id="myid", client_secret="mysecet")```)

The client ID and secret are provided by EDC. You will find them in your EDC Marketplace account section. You can also provide the credentials
via system environment variables (GEODB_AUTH_CLIENT_ID and GEODB_AUTH_CLIENT_SECRET). These variables can be supplied via a .env file.

## geoDB user groups

You may define user groups in geoDB, which serve the purpose to allow sharing of collections between project members while not making them publicly available. This notebook will demonstrate how to work with groups.

In [ ]:
from xcube_geodb.core.geodb import GeoDBClient

##  Login from your machine

Install xcube geoDB with the command:

    `conda install xcube_geodb -c conda-forge`

In [ ]:
# uncomment the following lines and fill in credentials
#client_id=YourID
#client_secret=YourSecret
#geodb = GeoDBClient(client_id=client_id, client_secret=client_secret, auth_mode="client-credentials", server_url='https://xcube-geodb.stage.brockmann-consult.de', auth_aud='https://xcube-users.brockmann-consult.de/api/v2')

## Login in EDC environment

In [ ]:
### comment if not on EDC

geodb = GeoDBClient()

Show current user, and the list of groups it is currently member of:

In [ ]:
print(geodb.whoami)
geodb.get_my_groups()

Create a new group, and checking before if it already exists (an error occurs if it does):

In [ ]:
group_name = 'test_group_' + geodb.whoami
group_name = group_name[:63] # ensure group_name will not be truncated in database
if group_name not in geodb.get_my_groups():
    geodb.create_group(group_name)


Add a user and see who's part of the group:

In [ ]:
geodb.add_user_to_group('geodb_thomas.storm.test', group_name)
geodb.get_group_users(group_name)

In [ ]:
geodb.get_my_groups()

Create a small test collection to be shared with the group:

In [ ]:
import geopandas

properties = {
    "RABA_PID": "float",
    "RABA_ID": "float",
    "D_OD": "date"
}

collection_name = 'land_use'
geodb.create_collection(collection_name, properties, crs=3794, clear=True)
gdf = geopandas.read_file('data/sample/land_use.shp')
m = geodb.insert_into_collection(collection_name, gdf.iloc[:100, :])
str(m)

Share new collection with group:

In [ ]:
m = geodb.publish_collection_to_group(collection_name, group_name)
str(m)

Show which users or groups have which rights on the collection:

In [ ]:
geodb.get_access_rights(collection_name)

Unpublish collection again:

In [ ]:
m = geodb.unpublish_collection_from_group(collection_name, group_name)
str(m)

Verify that groups does not have access to the collection anymore:

In [ ]:
geodb.get_access_rights(collection_name)

Finally, do a clean-up of this notebook's leftovers:

In [ ]:
geodb.drop_collection("land_use")